In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import requests
import json
from numpy import cos, sin, arcsin, sqrt
from math import radians
from datetime import datetime


from fuzzywuzzy import fuzz 
from pandas import ExcelWriter
import time
from pandas import ExcelFile

import shapely
from shapely.geometry import shape, mapping
import geopandas as gpd
from geopandas.tools import sjoin

from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

/Users/gowthamasokan/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/Users/gowthamasokan/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
MA_Block_group_WP = pd.read_csv("MA_Block_groups_Ward_Precinct_merge.csv")
MA_Block_group_WP

/Users/gowthamasokan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,ALAND10,AWATER10,...,PRECINCT,DISTRICT,POP_2010,TOWN,TOWN_ID,AREA_SQMI,AREA_ACRES_right,YEAR,SHAPE_AREA_right,SHAPE_LEN_right
0,0,25,23,525104,2,250235251042,Block Group 2,G5030,2648651.0,119260.0,...,4,4,3904,BRIDGEWATER,42,6.056,3875.80,2012,1.568486e+07,20641.267938
1,0,25,23,525104,2,250235251042,Block Group 2,G5030,2648651.0,119260.0,...,2,2,3886,BRIDGEWATER,42,2.961,1894.97,2012,7.668706e+06,12250.788877
2,0,25,23,525104,2,250235251042,Block Group 2,G5030,2648651.0,119260.0,...,5,5,3623,BRIDGEWATER,42,1.543,987.39,2012,3.995852e+06,9550.729218
3,0,25,23,525104,2,250235251042,Block Group 2,G5030,2648651.0,119260.0,...,1,1,3873,BRIDGEWATER,42,5.767,3691.10,2012,1.493741e+07,21003.563879
4,1,25,23,525104,4,250235251044,Block Group 4,G5030,4625818.0,11563.0,...,2,2,3886,BRIDGEWATER,42,2.961,1894.97,2012,7.668706e+06,12250.788877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20834,4978,25,17,358400,1,250173584001,Block Group 1,G5030,939683.0,0.0,...,9,9,3472,LEXINGTON,155,3.399,2175.67,2012,8.804656e+06,16846.619122
20835,4978,25,17,358400,1,250173584001,Block Group 1,G5030,939683.0,0.0,...,4,4,3475,LEXINGTON,155,1.293,827.23,2012,3.347697e+06,8666.973004
20836,4978,25,17,358400,1,250173584001,Block Group 1,G5030,939683.0,0.0,...,5,5,3526,LEXINGTON,155,1.828,1170.14,2012,4.735404e+06,10760.414353
20837,4978,25,17,358400,1,250173584001,Block Group 1,G5030,939683.0,0.0,...,8,8,3427,LEXINGTON,155,1.433,916.92,2012,3.710674e+06,8048.276445


In [4]:
list(MA_Block_group_WP.columns)

['Unnamed: 0',
 'STATEFP10',
 'COUNTYFP10',
 'TRACTCE10',
 'BLKGRPCE10',
 'GEOID10',
 'NAMELSAD10',
 'MTFCC10',
 'ALAND10',
 'AWATER10',
 'INTPTLAT10',
 'INTPTLON10',
 'AREA_SQFT',
 'AREA_ACRES_left',
 'POP100_RE',
 'HU100_RE',
 'LOGPL94171',
 'LOGSF1',
 'LOGACS0610',
 'LOGSF1C',
 'SHAPE_AREA_left',
 'SHAPE_LEN_left',
 'geometry',
 'index_right',
 'WP_NAME',
 'WARD',
 'PRECINCT',
 'DISTRICT',
 'POP_2010',
 'TOWN',
 'TOWN_ID',
 'AREA_SQMI',
 'AREA_ACRES_right',
 'YEAR',
 'SHAPE_AREA_right',
 'SHAPE_LEN_right']

In [6]:
Suffolk_County_CVAP = pd.read_excel("suffolk_county_block_groups.xlsx")
Suffolk_County_CVAP

,geoid,name,median_housing_price,pop,pop_whi,pop_bla,pop_his,pop_asi,cvap_total,cvap_asi,cvap_bla,cvap_whi,cvap_his
0,250250909012,"Block Group 2, Census Tract 909.01, Suffolk Co...",NaN,1925,676,162,345,680,990,50,205,555,125
1,250250907003,"Block Group 3, Census Tract 907, Suffolk Count...",383700.0,1185,652,89,267,120,745,35,80,535,70
2,250250921012,"Block Group 2, Census Tract 921.01, Suffolk Co...",387300.0,1004,247,57,73,619,810,380,65,195,130
3,250250002024,"Block Group 4, Census Tract 2.02, Suffolk Coun...",NaN,925,71,51,764,14,360,30,4,105,220
4,250251707021,"Block Group 1, Census Tract 1707.02, Suffolk C...",NaN,664,125,151,388,0,355,0,85,65,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,250250005022,"Block Group 2, Census Tract 5.02, Suffolk Coun...",389500.0,742,570,35,56,81,575,50,45,435,40
642,250250809001,"Block Group 1, Census Tract 809, Suffolk Count...",NaN,1265,553,33,104,575,870,90,135,470,120
643,250250912003,"Block Group 3, Census Tract 912, Suffolk Count...",384100.0,652,346,235,27,4,370,0,110,260,0
644,250250819002,"Block Group 2, Census Tract 819, Suffolk Count...",608500.0,813,38,560,192,23,640,0,415,30,195


In [9]:
Suffolk_County_CVAP['geoid']

0      250250909012
1      250250907003
2      250250921012
3      250250002024
4      250251707021
           ...     
641    250250005022
642    250250809001
643    250250912003
644    250250819002
645    250250910012
Name: geoid, Length: 646, dtype: int64

In [10]:
WP_BlockGroup_prop_area['GEOID10']

0        250235251042
1        250235251042
2        250235251042
3        250235251044
4        250235251044
             ...     
26035    250173417005
26036    250173417005
26037    250173418004
26038    250173418004
26039    250173418004
Name: GEOID10, Length: 26040, dtype: int64

In [4]:
WP_BlockGroup_prop_area = pd.read_csv("/Users/gowthamasokan/Geospatial Processing/MA_Block_groups_WP_area_proportional_merge.csv")
WP_BlockGroup_prop_area


/Users/gowthamasokan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,Unnamed: 0.1,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,ALAND10,...,TOWN_ID,AREA_SQMI,AREA_ACRES_2,YEAR,SHAPE_AREA_2,SHAPE_LEN_2,geometry,Tract_BlockGroup,Intersection_area,Percentage_of_Tract
0,0,0,25,23,525104,2,250235251042,Block Group 2,G5030,2648651.0,...,42,2.961,1894.97,2012,7.668706e+06,12250.788877,POLYGON ((-70.98036800024896 41.96639300025783...,5251042,0.000200,66.644304
1,1,12,25,23,525104,2,250235251042,Block Group 2,G5030,2648651.0,...,42,6.056,3875.80,2012,1.568486e+07,20641.267938,"POLYGON ((-70.97326399966906 41.9684209998961,...",5251042,0.000093,30.809348
2,2,16,25,23,525104,2,250235251042,Block Group 2,G5030,2648651.0,...,42,1.543,987.39,2012,3.995852e+06,9550.729218,POLYGON ((-70.96141799959049 41.97978900015519...,5251042,0.000008,2.546348
3,0,1,25,23,525104,4,250235251044,Block Group 4,G5030,4625818.0,...,42,2.961,1894.97,2012,7.668706e+06,12250.788877,POLYGON ((-71.00731199997252 41.96059700028049...,5251044,0.000493,97.853929
4,1,8,25,23,525104,4,250235251044,Block Group 4,G5030,4625818.0,...,42,5.767,3691.10,2012,1.493741e+07,21003.563879,POLYGON ((-71.00190600057279 41.95646799966004...,5251044,0.000011,2.146071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26035,0,9881,25,17,341700,5,250173417005,Block Group 5,G5030,113693.0,...,165,0.460,294.37,2012,1.191266e+06,5385.887098,POLYGON ((-71.05350199999367 42.43459899972473...,3417005,0.000009,75.511474
26036,1,10108,25,17,341700,5,250173417005,Block Group 5,G5030,113693.0,...,165,0.227,145.27,2012,5.878774e+05,3953.744584,POLYGON ((-71.04947300035781 42.43281399968268...,3417005,0.000003,24.488526
26037,0,8722,25,17,341800,4,250173418004,Block Group 4,G5030,260183.0,...,165,0.312,199.92,2012,8.090559e+05,5563.798922,POLYGON ((-71.05413700043901 42.42950999991532...,3418004,0.000015,54.340310
26038,1,8788,25,17,341800,4,250173418004,Block Group 4,G5030,260183.0,...,165,0.268,171.83,2012,6.953670e+05,4315.524552,POLYGON ((-71.05696200041314 42.42940200016659...,3418004,0.000003,8.916478


In [13]:
#Merged_CVAP_prop_area = pd.merge(WP_BlockGroup_prop_area, Suffolk_County_CVAP, left_on = 'GEOID10', right_on = 'geoid')
#Merged_CVAP_prop_area

#Merged_CVAP_prop_area = WP_BlockGroup_prop_area.join(Suffolk_County_CVAP, on=["GEOID10", "geoid"])
#Merged_CVAP_prop_area

Merged_CVAP_prop_area = pd.merge(WP_BlockGroup_prop_area, Suffolk_County_CVAP, left_on = 'GEOID10', right_on = 'geoid',how='left')
Merged_CVAP_prop_area



,Unnamed: 0,Unnamed: 0.1,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,ALAND10,...,pop,pop_whi,pop_bla,pop_his,pop_asi,cvap_total,cvap_asi,cvap_bla,cvap_whi,cvap_his
0,0,0,25,23,525104,2,250235251042,Block Group 2,G5030,2648651.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,12,25,23,525104,2,250235251042,Block Group 2,G5030,2648651.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,16,25,23,525104,2,250235251042,Block Group 2,G5030,2648651.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1,25,23,525104,4,250235251044,Block Group 4,G5030,4625818.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,8,25,23,525104,4,250235251044,Block Group 4,G5030,4625818.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26035,0,9881,25,17,341700,5,250173417005,Block Group 5,G5030,113693.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26036,1,10108,25,17,341700,5,250173417005,Block Group 5,G5030,113693.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26037,0,8722,25,17,341800,4,250173418004,Block Group 4,G5030,260183.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26038,1,8788,25,17,341800,4,250173418004,Block Group 4,G5030,260183.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
Merged_CVAP_prop_area[120:180]

,Unnamed: 0,Unnamed: 0.1,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,ALAND10,...,pop,pop_whi,pop_bla,pop_his,pop_asi,cvap_total,cvap_asi,cvap_bla,cvap_whi,cvap_his
120,2,1112,25,23,510400,1,250235104001,Block Group 1,G5030,184884.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,0,55,25,23,510300,2,250235103002,Block Group 2,G5030,140889.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,1,406,25,23,510300,2,250235103002,Block Group 2,G5030,140889.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,0,56,25,23,510400,2,250235104002,Block Group 2,G5030,174390.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,1,411,25,23,510400,2,250235104002,Block Group 2,G5030,174390.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,2,1129,25,23,510400,2,250235104002,Block Group 2,G5030,174390.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,0,57,25,23,511000,1,250235110001,Block Group 1,G5030,379354.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,1,412,25,23,511000,1,250235110001,Block Group 1,G5030,379354.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,2,418,25,23,511000,1,250235110001,Block Group 1,G5030,379354.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,0,52,25,23,510200,2,250235102002,Block Group 2,G5030,768899.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
Merged_CVAP = pd.merge(MA_Block_group_WP, Suffolk_County_CVAP, left_on = 'GEOID10', right_on = 'geoid')
Merged_CVAP

# Merging the MA_Block_group with the Suffolk CVAP

,Unnamed: 0,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,ALAND10,AWATER10,...,pop,pop_whi,pop_bla,pop_his,pop_asi,cvap_total,cvap_asi,cvap_bla,cvap_whi,cvap_his
0,1432,25,25,100500,2,250251005002,Block Group 2,G5030,187391.0,0.0,...,840,357,179,162,46,715,0,115,470,130
1,1432,25,25,100500,2,250251005002,Block Group 2,G5030,187391.0,0.0,...,840,357,179,162,46,715,0,115,470,130
2,1432,25,25,100500,2,250251005002,Block Group 2,G5030,187391.0,0.0,...,840,357,179,162,46,715,0,115,470,130
3,1432,25,25,100500,2,250251005002,Block Group 2,G5030,187391.0,0.0,...,840,357,179,162,46,715,0,115,470,130
4,1432,25,25,100500,2,250251005002,Block Group 2,G5030,187391.0,0.0,...,840,357,179,162,46,715,0,115,470,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2377,3444,25,25,170702,4,250251707024,Block Group 4,G5030,155395.0,0.0,...,1474,243,0,883,263,370,40,0,155,175
2378,3444,25,25,170702,4,250251707024,Block Group 4,G5030,155395.0,0.0,...,1474,243,0,883,263,370,40,0,155,175
2379,3444,25,25,170702,4,250251707024,Block Group 4,G5030,155395.0,0.0,...,1474,243,0,883,263,370,40,0,155,175
2380,3445,25,25,40801,3,250250408013,Block Group 3,G5030,412850.0,338688.0,...,2314,2008,24,172,95,2075,60,20,1940,35


In [5]:
Merged_CVAP[['TRACTCE10','BLKGRPCE10', 'SHAPE_AREA_left', 'SHAPE_AREA_right','pop','TOWN','WARD','PRECINCT']][0:15]

,TRACTCE10,BLKGRPCE10,SHAPE_AREA_left,SHAPE_AREA_right,pop,TOWN,WARD,PRECINCT
0,100500,2,187377.333347,102375.388218,840,BOSTON,17,11
1,100500,2,187377.333347,336499.534370,840,BOSTON,16,8
2,100500,2,187377.333347,226457.504223,840,BOSTON,17,9
3,100500,2,187377.333347,318539.112571,840,BOSTON,17,8
4,100500,2,187377.333347,264112.564034,840,BOSTON,16,6
5,100500,2,187377.333347,254746.448735,840,BOSTON,16,3
6,100500,2,187377.333347,277616.037474,840,BOSTON,17,6
7,100601,4,139107.023155,264112.564034,2183,BOSTON,16,6
8,100601,4,139107.023155,190330.285792,2183,BOSTON,16,4
9,100601,3,108556.655382,285883.619875,1421,BOSTON,16,7


In [5]:
Zip_Census_proportion = pd.read_excel("ZIP_TRACT_032021.xlsx")
Zip_Census_proportion

,ZIP,TRACT,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,1001,25013813209,AGAWAM,MA,0.404209,0.436716,0.249027,0.402993
1,1001,25013813208,AGAWAM,MA,0.149738,0.044470,0.038911,0.136825
2,1001,25013813204,AGAWAM,MA,0.000487,0.000000,0.000000,0.000428
3,1001,25013813207,AGAWAM,MA,0.228318,0.259977,0.509728,0.239017
4,1001,25013813206,AGAWAM,MA,0.012651,0.005701,0.003891,0.011758
...,...,...,...,...,...,...,...,...
2913,2790,25005640100,WESTPORT,MA,0.001086,0.000000,0.000000,0.000989
2914,2790,25005653304,WESTPORT,MA,0.001846,0.001362,0.000000,0.001780
2915,2790,25005646104,WESTPORT,MA,0.260667,0.200272,0.183432,0.254993
2916,2791,25005646104,WESTPORT POINT,MA,0.000000,0.000000,1.000000,1.000000
